# Chapter 7.1.0 - Properties

In [3]:
class Person:
    def __init__(self,firstName,lastName):
        self.firstName = firstName
        self.lastName = lastName

    @property # property decorator - this function is only a readable property now
    def fullName(self):
        return f"{self.firstName} {self.lastName}"

johnWalter = Person('John','Walter')
johnWalter.fullName

'John Walter'

In [6]:
class Person:
    def __init__(self,firstName,lastName):
        self.firstName = firstName
        self.lastName = lastName

    @property 
    def fullName(self):
        return f"{self.firstName} {self.lastName}"

    @fullName.setter
    def fullName(self,value):
       self.firstName,self.lastName = value.split(' ',1)

kandy = Person('Kandy','Woods')
kandy.fullName 
#After marriage
kandy.fullName = 'Kandy Walker'
kandy.fullName

'Kandy Walker'

In [9]:
# back public readable variables by no-readable ones
class Amount:
    def __init__(self,amount):
        self._amount = amount # javascript also add an underscore to "private properties"    
    @property
    def amount(self):
        return self._amount 

cost = Amount(1.23)
display(cost.amount)
cost.amount = 1.50

1.23

AttributeError: property 'amount' of 'Amount' object has no setter

In [12]:
class Ticket:
    def __init__(self,price):
        self.price = price     
    @property
    def price(self):
        return self._price
    @price.setter
    def price(self,value):
        if value < 0:
            raise ValueError('Not in my house')
        self._price = value 

tygaTickets = Ticket(237)
tygaTickets.price = 200
tygaTickets.price


200

In [13]:
tygaTickets.price = -50

ValueError: Not in my house

# Section 7.1.1 - Properties and Refactoring

In [1]:
# Imagine Before
# class Money:
#     def __init__(self,dollars,cents):
#         self.dollars = dollars
#         self.cents = cents 

# And After
class Money:
    def __init__(self,dollars,cents):
        self.total_cents = dollars * 100 + cents 

In [2]:
#code written like this during the before will break
myWallet = Money(500,32)
myWallet.price

AttributeError: 'Money' object has no attribute 'price'

In [21]:
# To fix, properties are used
class Money:
    def __init__(self,dollars,cents):
        self.total_cents = dollars * 100 + cents 

    @property
    def dollars(self):
        return self.total_cents / 100)
    
    @dollars.setter
    def dollars(self,newDollars):
        self.total_cents = 100 * newDollars + self.cents

    @property
    def cents(self):
        return self.total_cents % 100 
    
    @cents.setter
    def dollars(self,newCents):
        self.total_cents = newCents + self.dollars * 100

myMoney = Money(50,0)

    

SyntaxError: unmatched ')' (147155631.py, line 8)

In [22]:
display(myMoney.cents,
myMoney.dollars,
myMoney.total_cents
)

50

50

50

In [20]:
myMoney.dollars

50

# Section 7.2 The Factory Patterns

### Section 7.2.1 Alternative Constructors: The Simple Factory

In [32]:
# can use class methods to handle creating the same object in different variations

class Money:
    def __init__(self,dollars,cents):
        self.dollars = dollars
        self.cents = cents 
    @classmethod
    def moneyFromPennies(cls,pennies):
        dollars = pennies // 100
        cents = pennies % 100
        return cls(dollars,cents)
    @classmethod
    def moneyFromQuarters(cls,quarters):
        dollars = quarters // 4
        cents = quarters % 4 * 25
        return cls(dollars,cents)

regularMoney = Money(5,25)
pennies = Money.moneyFromPennies(525)
quarters = Money.moneyFromQuarters(21)
print(f"Regular money in dollars is {regularMoney.dollars} and {regularMoney.cents} cents")
print(f"pennies in dollars is {pennies.dollars} and {pennies.cents} cents")
print(f"quarters in dollars is {quarters.dollars} and {quarters.cents} cents")

Regular money in dollars is 5 and 25 cents
pennies in dollars is 5 and 25 cents
quarters in dollars is 5 and 25 cents


# Section 7.2.2 Dynamic Type: The Factory Method Pattern

In [33]:
import abc 
class ImageReader(metaclass=abc.ABCMeta):
    def __init__(self,path):
        self.path = path
    @abc.abstractmethod
    def read(self):
        pass #subclass must implement
    def __repr__(self):
        return f"{self.__class__.__name__}({self.path})"

class GIFReader(ImageReader):
    def read(self):
        "Read a GIF"

class JPEGReader(ImageReader):
    def read(self):
        "Read a JPEG"

class PNGReader(ImageReader):
    def read(self):
        "Read a PNG"



In [36]:
def extension_of(path):
    position_of_last_dot = path.rfind('.')
    return path[position_of_last_dot+1:].lower()

extension_of('document.csv')

'csv'

In [37]:
READERS = {
    "gif":GIFReader,
    "png":PNGReader,
    "jpeg":JPEGReader
}

def get_image_reader(path):
    try:
        reader_class = READERS[extension_of(path)]
        return reader_class(path)
    except KeyError:
        raise KeyError('This extension type is not supported')

get_image_reader('BeachPic.jpeg')

JPEGReader(BeachPic.jpeg)

In [39]:
# Can also do a default byte reader

class RawByteReader(ImageReader):
    def read(self):
        "Default Image Reader"

def get_image_reader(path):
    try:
        reader_class = READERS[extension_of(path)]
        return reader_class(path)
    except KeyError:
        return RawByteReader(path)

get_image_reader('RGBValues.csv')

RawByteReader(RGBValues.csv)

# Section-7.3 The Observer Pattern

In [41]:
# Section 7.3.1 The simple observer 

class Subscriber:
    def __init__(self,name):
        self.name = name 
    def update(self,message):
        print(f"{self.name} got message {message}")

In [42]:
class Publisher:
    def __init__(self):
        self.subscribers = set()
    def register(self,who):
        self.subscribers.add(who)
    def unregister(self,who):
        self.subscribers.discard(who)
    def dispatch(self,message):
        for subscriber in self.subscribers:
            subscriber.update(message)

In [43]:
pub = Publisher()
bob = Subscriber('Bob')
anna = Subscriber('Anna')
peter = Subscriber('Peter')

pub.register(bob)
pub.register(anna)
pub.register(peter)

In [45]:
pub.dispatch("it's lunchtime")

Anna got message it's lunchtime
Bob got message it's lunchtime
Peter got message it's lunchtime


In [46]:
# Section 7.3.2 A Pythonic Refinement

class SubscriberOne:
    def __init__(self,name):
        self.name = name
    def update(self, message):
        print(f'{self.name} got message "{message}" ')


class SubscriberTwo:
    def __init__(self,name):
        self.name = name
    def receive(self, message):
        print(f'{self.name} got message "{message}" ')

In [54]:
class Publisher:
    def __init__(self):
        self.subscribers = dict()
    def register(self, who, callback=None):
        if callback is None:
            callback = who.update
        self.subscribers[who] = callback
    def dispatch(self, message):
        for callback in self.subscribers.values():
            callback(message) 
    def unregister(self, who):
        del self.subscribers[who]


In [55]:
pub = Publisher()
jane = SubscriberOne('Jane')
ashley = SubscriberTwo('Ashley')
joan = SubscriberOne('Joan')

pub.register(jane,print) #doesn't have to be object method
pub.register(ashley,ashley.receive)
pub.register(joan,joan.update)


In [56]:
pub.dispatch("I'm Hungry")

I'm Hungry
Ashley got message "I'm Hungry" 
Joan got message "I'm Hungry" 


In [57]:
# 7.3.3 Several Channels
channels = ['lunch','dinner']
result = {channel : dict() 
            for channel in channels }

In [64]:
result['lunch']['peter'] = print

In [66]:
#lol
result['lunch']['peter']('hello world')

hello world


In [118]:
class Publisher:
    def __init__(self,channels):
        self.channels = {channel : dict()
            for channel in channels}

    def register(self,channel,who,callback=None):
        if callback is None:
            callback = who.update
        self.channels[channel][who] = callback 
    
    def dispatch(self, channel, message):
        subscribers = self.channels[channel]
        for callback in iter(subscribers.values()):
            callback(message)
        

pub = Publisher(['day','night'])
karl = SubscriberOne('karl') 
carol = SubscriberOne('carol')
jason = SubscriberOne('jason')


pub.register('day',carol)
pub.register('night',karl)
pub.register('night',jason)   



In [119]:
pub.dispatch('night','time for bed')


karl got message "time for bed" 
jason got message "time for bed" 


# Section 7.4 Magic Methods

In [125]:
class Angle:
    def __init__(self, value):
        self.value = value%360


102

In [140]:
#https://www.youtube.com/watch?v=OyGPockrCgA
result = -90 % 360
print(result)


270


In [152]:
# Section 7.4.1 simple math magic
class Angle:
    def __init__(self, value):
        self.value = value%360
    def __add__(self,other):
        return Angle(self.value + other.value)
    def __str__(self):
        return f"{self.value} degrees"
    def __repr__(self):
        return f"{self.value}"
    def __ne__(self,other):
        return self.value != other.value
    def __gt__(self,other):
        return self.value > other.value

total = Angle(40) + Angle(-500)
total.value

260

In [153]:
print(total)
display(str(total), total)

260 degrees


'260 degrees'

260

In [155]:
# work because dunder gt method
total > Angle(17)

True

In [157]:
# Section 7.4.1.1 functools.total_ordering 
import functools

@functools.total_ordering
class Angle:
    def __init__(self,value):
        self.value = value%360
    def __eq__(self,other):
        return self.value == other.value
    def __gt__(self,other):
        return self.value > other.value



In [159]:
Angle(270) < Angle(-89) # did not have to define __lt__

True